# Movie ratings database

.|OUTi...H | Parasite | Little Women | 1917
-|--------------------------------|----------|--------------|------
alice | 1 | 1 | 0 | 1
  bob | 0 | 1 | 1 | 0
carol | 1 | 1 | 0 | 0
maria | 1 | 0 | 1 | 1

> "OUTi...H" = Once Upon a Time in ... Hollywood

## SQL

Here is a natural way to store the above data in a SQL database.

In [ ]:
import sqlite3 as db
from pandas import read_sql_query
conn = db.connect('movie-ratings.db')
cursor = conn.cursor()

In [ ]:
cursor.execute("DROP TABLE IF EXISTS Users")
cursor.execute("CREATE TABLE Users (id INTEGER, name TEXT)")
cursor.executemany("INSERT INTO Users VALUES (?, ?)", [(0, 'alice'), (1, 'bob'), (2, 'carol'), (3, 'maria')])
conn.commit()
read_sql_query("SELECT * FROM Users", conn)

In [ ]:
cursor.execute("DROP TABLE IF EXISTS Movies")
cursor.execute("CREATE TABLE Movies (id INTEGER, name TEXT)")
cursor.executemany("INSERT INTO Movies VALUES (?, ?)", [(0, 'outi...h'), (1, 'parasite'), (2, 'little women'), (3, '1917')])
conn.commit()
read_sql_query("SELECT * FROM Movies", conn)

.|OUTi...H | Parasite | Little Women | 1917
-|--------------------------------|----------|--------------|------
alice | 1 | 1 | 0 | 1
  bob | 0 | 1 | 1 | 0
carol | 1 | 1 | 0 | 0
maria | 1 | 0 | 1 | 1

In [ ]:
cursor.execute("DROP TABLE IF EXISTS Ratings")
cursor.execute("CREATE TABLE Ratings (user INTEGER, movie INTEGER)")
cursor.executemany("INSERT INTO Ratings VALUES (?, ?)", [(0, 0), (0, 1), (0, 3),
                                                         (1, 1), (1, 2),
                                                         (2, 0), (2, 1),
                                                         (3, 0), (3, 2), (3, 3)])
conn.commit()
read_sql_query("SELECT * FROM Ratings", conn)

**Exercise:** Which user is "most similar" to Maria?

> _Answer:_ `alice`: 2 movies in common, `bob`: 1, `carol`: 1. Let's write some SQL to calculate it.

In the implementation below, for clarity, we'll use views to create some logical tables that we can then join to perform the desired comparisons.

In [ ]:
# Maria's viewed-movie list
cursor.execute("DROP VIEW IF EXISTS Maria")
cursor.execute("CREATE VIEW Maria AS SELECT movie FROM Ratings WHERE user=3")
read_sql_query("SELECT * FROM Maria", conn)

In [ ]:
# Everyone but Maria
cursor.execute("DROP VIEW IF EXISTS Everyone")
cursor.execute("CREATE VIEW Everyone AS SELECT user, movie FROM Ratings WHERE user<>3")
read_sql_query("SELECT * FROM Everyone", conn)

In [ ]:
read_sql_query("""
SELECT Everyone.user, Count(*)
  FROM Everyone, Maria
  WHERE Everyone.movie = Maria.movie
  GROUP BY user
""", conn)

> Here is a second version that merges in the names to make the output a little more readable.

In [ ]:
read_sql_query("""
SELECT Everyone.user, Users.name, Count(*)
  FROM Users, Everyone, Maria
  WHERE Everyone.movie = Maria.movie AND Users.id = Everyone.user
  GROUP BY user
""", conn)

In [ ]:
conn.close()

## Python version

Recall the dataset:

.|OUTi...H | Parasite | Little Women | 1917
-|--------------------------------|----------|--------------|------
alice | 1 | 1 | 0 | 1
  bob | 0 | 1 | 1 | 0
carol | 1 | 1 | 0 | 0
maria | 1 | 0 | 1 | 1

Here is one encoding of that table using a dictionary of dictionaries, with the outermost keys being the names (row labels) and the innermost keys being the movies (column labels). The convention in this case is to _omit_ any zero entries, thereby exploiting the _sparsity_ of the table.

In [ ]:
viewers = {'alice': {'outi...h': 1, 'parasite': 1, '1917': 1},
           'bob': {'parasite': 1, 'little women': 1},
           'carol': {'outi...h': 1, 'parasite': 1},
           'maria': {'outi...h': 1, 'little women': 1, '1917': 1}}
viewers

Let's develop a Python analogue of the SQL query from above.

In [ ]:
maria = viewers['maria']
maria

In [ ]:
everyone = {name: viewed for name, viewed in viewers.items() if name != 'maria'}
everyone

**Version 0.**

In [ ]:
from collections import defaultdict
similarities = defaultdict(int)
for name in everyone:
    for movie in everyone[name]:
        if movie in maria:
            similarities[name] += 1
similarities

**Version 1.** Same as above, but the innermost loop iterates over Maria's movies rather than the movies in `everyone[name]`. There is no intrinsic advantage or disadvantage between Version 0 or 1, unless we believe that Version that `maria` or `everyone[name]` will tend to have fewer nonzeros. (A method that wanted to minimize data items touched might iterate over whichever structure, `maria` or `everyone[name]`, has the fewest nonzeros.)

In [ ]:
from collections import defaultdict
similarities = defaultdict(int)
for name in everyone:
    for movie in maria:
        if movie in everyone[name]:
            similarities[name] += 1
similarities

**Version 2.** This version is the same as Version 0 but with the explicit `if` condition eliminated by the dictionary's `.get()` method.

In [ ]:
similarities = defaultdict(int)
for name in everyone:
    for movie in everyone[name]:
        similarities[name] += maria.get(movie, 0)
similarities

**Version 3.** In this version, we generalize the code slightly in the case that the values might not be just 0 and 1, but maybe ratings on a scale from, say, 0 to 4 "stars." In this instance, one might take the _product_ of two ratings to be a measure of similarity.

In [ ]:
similarities = defaultdict(int)
for name in everyone:
    for movie in everyone[name]:
        similarities[name] += everyone[name][movie] * maria.get(movie, 0)
similarities

_Don't do the following!_ Remember that accessing a default dictionary with _any_ key will create an entry for that key, even if you do not do assignment! Example:

In [ ]:
similarities['rich']
similarities

## Connecting ideas

Let
- $y$ denote `similarities`;
- $i$ denote `name`;
- $x$ denote `maria`;
- $j$ denote `movie`; and
- $A$ denote `everyone`.

Then body of above loop is

$$y_i \leftarrow y_i + a_{i,j} \cdot x_j,$$

which is an "update equation," since it implies some kind of looping structure that updates the value $y_i$. In this case, if we further assume that $y_i$ is 0 initially, which would be the case in counting, then you should recognize this update as a _matrix-vector multiplication_ (per Topic 3):

$$\implies y = A \cdot x$$

This equation is the _matrix form._ It is equivalent to the following _scalar form_, for all $y_i$:
$$\implies y_i = \sum_{j} a_{i,j} \cdot x_j.$$

From the scalar form, the most natural implementation would, at first glance, seem to be something like,

```python
for i in range(m):
    for j in range(n):
        y[i] += A[i, j] * x[j]
```

The above loop-nest is a straightforward translation of the mathematical definition of matrix-vector multiplication. But it's not necessarily efficient in this example because it _ignores_ sparsity! That is, it visits every entry of $A$ even though our example (if you imagined many millions of users and hundreds of thousands of movies) would be quite sparse. You should review the Python implementations above to see how the choice of data structure affects the choice of how to efficiently implement the loops.

> In the SQL example, it's arguably less clear whether sparsity is exploited appropriately because it is, as we've discussed, _declarative_, meaning it says what to compute but does not dictate how. By contrast, the Python version is an _imperative_ program: it prescribes exactly how to execute the loops.